Домашнее задание №2 по предмету NLP

In [33]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, GRU, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import accuracy_score, f1_score
from sklearn.utils import resample
import re

In [34]:
# Чтение данных
df = pd.read_csv("C:/Users/Petroo/Desktop/train.csv")

In [35]:
# Преобразовываем данные к строковому типу и заменяем NaN на пустые строки
df['Text'] = df['Text'].astype(str)
df['Text'] = df['Text'].fillna('')  # проверяем что нет пустых значений

In [36]:
# Удаляем дубликаты
df.drop_duplicates(subset='Text', inplace=True)

In [37]:
# Очищаем данные и  приводим к нижнему регистру
def clean_text(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)  
    text = re.sub(r'[^\w\s]', '', text)  
    text = re.sub(r'\s+', ' ', text).strip()  
    return text

df['Text'] = df['Text'].apply(clean_text)  # Применяем очистку

In [38]:
# Балансируем классы с использованием ресэмплинга
df_majority = df[df['Sentiment'] == 'Neutral']
df_minority_negative = df[df['Sentiment'] == 'Negative']
df_minority_positive = df[df['Sentiment'] == 'Positive']

df_minority_negative_upsampled = resample(df_minority_negative, replace=True, n_samples=len(df_majority), random_state=42)
df_minority_positive_upsampled = resample(df_minority_positive, replace=True, n_samples=len(df_majority), random_state=42)

# Создаем сбалансированный набор данных
df_balanced = pd.concat([df_majority, df_minority_negative_upsampled, df_minority_positive_upsampled])


In [39]:
# Подготовка текстовых данных и целевых меток
X = df_balanced['Text'].values
y = df_balanced['Sentiment'].values

In [40]:
# Кодируем целевые метки с использованием LabelEncoder
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)  

In [41]:
# Токенизируем текст и приводим к фиксированной длине
MAX_LEN = 100  
tokenizer = Tokenizer(num_words=20000, oov_token="<OOV>")  
tokenizer.fit_on_texts(X)
X_sequences = tokenizer.texts_to_sequences(X)  
X_padded = pad_sequences(X_sequences, maxlen=MAX_LEN, padding='post')  

In [42]:
# Делим данные на  обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X_padded, y_encoded, test_size=0.2, random_state=42)

In [43]:
# Определение моделей 
def create_lstm_model(max_vocab_size, max_len, num_classes):
    model = Sequential()
    model.add(Embedding(input_dim=max_vocab_size + 1, output_dim=128))  # Слой Embedding
    model.add(Bidirectional(LSTM(64)))  # Bidirectional LSTM слой
    model.add(Dropout(0.5))  # Dropout для предотвращения переобучения
    model.add(Dense(num_classes, activation='softmax'))  # Полносвязный слой для классификации
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])  # Компиляция модели
    return model

def create_gru_model(max_vocab_size, max_len, num_classes):
    model = Sequential()
    model.add(Embedding(input_dim=max_vocab_size + 1, output_dim=128))
    model.add(Bidirectional(GRU(64)))  # Bidirectional GRU слой
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

def create_bilstm_model(max_vocab_size, max_len, num_classes):
    model = Sequential()
    model.add(Embedding(input_dim=max_vocab_size + 1, output_dim=128))
    model.add(Bidirectional(LSTM(64)))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


In [44]:
# Обучение моделей
num_classes = len(label_encoder.classes_)  
models = {
    'LSTM': create_lstm_model(len(tokenizer.word_index), MAX_LEN, num_classes),
    'GRU': create_gru_model(len(tokenizer.word_index), MAX_LEN, num_classes),
    'Bidirectional LSTM': create_bilstm_model(len(tokenizer.word_index), MAX_LEN, num_classes)
}


In [45]:
# Оценка моделей
results = {}

for model_name, model in models.items():
    print(f'Обучение модели {model_name}...')
    history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1)  # Обучение модели
    
    # Оценка модели
    y_pred = model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)  # Преобразуем предсказания в классы
    accuracy = accuracy_score(y_test, y_pred_classes)  # Точность
    f1 = f1_score(y_test, y_pred_classes, average='weighted')  # F1-score
    
    results[model_name] = {'Accuracy': accuracy, 'F1 Score': f1}  # Сохраняем результат
    print(f'{model_name} Accuracy: {accuracy}')
    print(f'{model_name} F1 Score: {f1}')


Обучение модели LSTM...
Epoch 1/10
521/521 ━━━━━━━━━━━━━━━━━━━━ 55s 98ms/step - accuracy: 0.4785 - loss: 0.9959 - val_accuracy: 0.7202 - val_loss: 0.6970
Epoch 2/10
521/521 ━━━━━━━━━━━━━━━━━━━━ 52s 99ms/step - accuracy: 0.8351 - loss: 0.4757 - val_accuracy: 0.7893 - val_loss: 0.5846
Epoch 3/10
521/521 ━━━━━━━━━━━━━━━━━━━━ 51s 98ms/step - accuracy: 0.9117 - loss: 0.2690 - val_accuracy: 0.7915 - val_loss: 0.6047
Epoch 4/10
521/521 ━━━━━━━━━━━━━━━━━━━━ 51s 98ms/step - accuracy: 0.9421 - loss: 0.1815 - val_accuracy: 0.7958 - val_loss: 0.7132
Epoch 5/10
521/521 ━━━━━━━━━━━━━━━━━━━━ 52s 99ms/step - accuracy: 0.9646 - loss: 0.1080 - val_accuracy: 0.8017 - val_loss: 0.7591
Epoch 6/10
521/521 ━━━━━━━━━━━━━━━━━━━━ 50s 97ms/step - accuracy: 0.9748 - loss: 0.0850 - val_accuracy: 0.7915 - val_loss: 0.9614
Epoch 7/10
521/521 ━━━━━━━━━━━━━━━━━━━━ 51s 98ms/step - accuracy: 0.9818 - loss: 0.0630 - val_accuracy: 0.7958 - val_loss: 0.9537
Epoch 8/10
521/521 ━━━━━━━━━━━━━━━━━━━━ 52s 99ms/step - accuracy: 

In [46]:
# Вывод результатов для всех моделей
print("\nРезультаты всех моделей:")
for model_name, metrics in results.items():
    print(f"{model_name}: Accuracy = {metrics['Accuracy']}, F1 Score = {metrics['F1 Score']}")



Результаты всех моделей:
LSTM: Accuracy = 0.7709098768100281, F1 Score = 0.7704608364427223
GRU: Accuracy = 0.7817160146963475, F1 Score = 0.7818124982703244
Bidirectional LSTM: Accuracy = 0.7944672574022045, F1 Score = 0.7943969280894562


Вывод

В процессе решения задачи классификации тональности текста были обучены и сравнены различные подходы: классические модели машинного обучения, эвристические методы, и RNN.

Эвристические методы

Данный подход может показать адекватные результаты для небольших задач или в случае ограниченного объема данных, его эффективность оказалась значительно ниже по сравнению с машинным обучением. Главным недостатком является не учёт контекста и сложных зависимостей в тексте.

Классические модели машинного обучения

Результаты по сравнению с эвристикой получились лучше, благодаря использованию методов векторизации текста, таких как TF-IDF, но эти модели также имеют ограничения в части более сложных языковых конструкций и синтаксиса.

Рекуррентные нейронные сети (RNN)

RNN (LSTM, GRU и Bidirectional LSTM) показали наилучшие результаты, так как они способны учитывать порядок слов, контекст и долгосрочные зависимости в тексте. Эти модели обучались на токенизированных данных, что позволило захватить сложные зависимости между словами, и их производительность была значительно выше по сравнению с эвристиками и классическими моделями.
Несмотря на то, что LSTM и GRU показали близкие результаты, Bidirectional LSTM позволила моделям более точно учитывать и обратный контекст, что важно может быть важно для некоторых типов текстов.

RNN продемонстрировали наибольшую точность по сравнению с эвристиками и классическими моделями машинного обучения, учитывая их способность обрабатывать последовательные данные и учитывать длинные зависимости в тексте. 
Но для них потребовалось гораздо больше вычислительных ресурсов и времени на обучение.

Эвристические методы могут быть полезны в условиях ограниченных вычислительных мощностей или быстрого анализа, но они очень уступают современным методам. Классические модели машинного обучения обеспечивают баланс между скоростью и точностью, но с усложнением данных RNN оказываются наиболее эффективным решением.